In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [3]:
import zipfile
import os 

zip_path = "DATASET-F.zip"  # Path to your ZIP file
extract_path = "WASTE-CLASS"  # Destination folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [4]:
print(os.listdir("WASTE-CLASS\DATASET-F"))  # Should show 'train' and 'test' folders


['img_rename.py', 'test', 'train']


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_data = datagen.flow_from_directory('WASTE-CLASS/DATASET-F/train',
    target_size=(224,224), batch_size=32, class_mode='categorical')

# Load testing data
test_data = datagen.flow_from_directory('WASTE-CLASS/DATASET-F/test',
    target_size=(224,224), batch_size=32, class_mode='categorical')


Found 612 images belonging to 3 classes.
Found 167 images belonging to 3 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

# Load MobileNetV2 as a feature extractor
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # Adjust for your number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,691 (9.87 MB)

 Trainable params: 328,707 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
!pip install --upgrade pillow

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Use test_data instead of val_data
model.fit(train_data, epochs=10)  # Remove validation_data argument



d:\Ishani\FINAL YEAR PROJECT\STAGE2-WORKING\dataset-f\waste_classification-FLASK\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 611ms/step - accuracy: 0.6133 - loss: 0.9348
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 606ms/step - accuracy: 0.9287 - loss: 0.1861
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 603ms/step - accuracy: 0.9656 - loss: 0.0896
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 650ms/step - accuracy: 0.9876 - loss: 0.0613
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 654ms/step - accuracy: 0.9978 - loss: 0.0302
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 593ms/step - accuracy: 0.9937 - loss: 0.0250
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 583ms/step - accuracy: 1.0000 - loss: 0.0181
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 586ms/step - accuracy: 1.0000 - loss: 0.0107
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 590ms/step - accuracy: 1.0000 - loss: 0.0081
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 582ms/step - accuracy: 1.0000 - loss: 0.0076


In [9]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 527ms/step - accuracy: 0.9484 - loss: 0.1532
Test Accuracy: 93.41%
Test Loss: 0.2405


In [8]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 506ms/step - accuracy: 0.9563 - loss: 0.1632
Test Accuracy: 95.76%
Test Loss: 0.1733


In [ ]:
model.save("waste_classifier.keras")

In [11]:
model.save("waste_classifier.h5")

In [12]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("waste_classifier.h5")

# Load and preprocess image
image_path = "WASTE-CLASS/DATASET-F/train/Wet/wet_002.jpg"  # Change to your image path
img = load_img(image_path, target_size=(224, 224))  # Resize
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(img_array)
class_labels = ["Dry", "Paper", "Wet"]
predicted_class = class_labels[np.argmax(predictions)]  # Get highest probability class
print(f"Predicted Class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: Wet


In [13]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("waste_classifier.h5")

# Load and preprocess image
image_path = "WASTE-CLASS/DATASET-F/train/Paper/paper_007.jpg"  # Change to your image path
img = load_img(image_path, target_size=(224, 224))  # Resize
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(img_array)
class_labels = ["Dry", "Paper", "Wet"]
predicted_class = class_labels[np.argmax(predictions)]  # Get highest probability class
print(f"Predicted Class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: Paper


In [14]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("waste_classifier.h5")

# Load and preprocess image
image_path = "WASTE-CLASS/DATASET-F/test/Dry/dry_006.jpg"  # Change to your image path
img = load_img(image_path, target_size=(224, 224))  # Resize
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(img_array)
class_labels = ["Dry", "Paper", "Wet"]
predicted_class = class_labels[np.argmax(predictions)]  # Get highest probability class
print(f"Predicted Class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Dry
